In [1]:
import langchain
import os

In [23]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import ctransformers
import pinecone
from pinecone import Pinecone as PineconeClient, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
from langchain_openai import ChatOpenAI 

In [ ]:
PINECONE_API_KEY = "***********************"
PINECONE_API_ENV = "******"

In [4]:
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY
os.environ['PINECONE_API_ENV'] = PINECONE_API_ENV

In [5]:
def load_pdf(data):
    loader = DirectoryLoader(data,
                             glob='*.pdf',
                             loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [9]:
extracted_data = load_pdf("data/")

KeyboardInterrupt: 

In [7]:
def text_split(extracted_data): 
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [8]:
text_chunks = text_split(extracted_data)
print("length of my chunks:",len(text_chunks))

NameError: name 'extracted_data' is not defined

In [12]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [14]:
embeddings = download_hugging_face_embeddings()

In [15]:
embeddings

HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False)

In [16]:
index_name = "medical-chatbot"
docsearch = PineconeVectorStore.from_texts(
        texts=[t.page_content for t in text_chunks],
        index_name=index_name,
        embedding=embeddings
    )

NameError: name 'text_chunks' is not defined

In [17]:
index_name = "medical-chatbot"
docsearch = PineconeVectorStore.from_existing_index(index_name, embeddings)

query = "What are Allergies"

docs = docsearch.similarity_search(query, k=3)

print("Result",docs)

Result [Document(id='66a4849a-a98a-4305-9bbd-ee8cf120cf6e', metadata={}, page_content='‘‘What’s New in: Asthma and Allergic Rhinitis.’’Pulse\n(September 20, 2004): 50.\nRichard Robinson\nTeresa G. Odle\nAllergies\nDefinition\nAllergies are abnormal reactions of the immune\nsystem that occur in response to otherwise harmless\nsubstances.\nDescription\nAllergies are among the most common of medical\ndisorders. It is estimated that 60 million Americans, or\nmore than one in every five people, suffer from some\nform of allergy, with similar proportions throughout'), Document(id='2abd89e1-a4ad-4896-a07e-afce9d23f5cb', metadata={}, page_content='KEY TERMS\nAllergen— A substance capable of inducing an\nallergic response.\nAllergic reaction— An immune system reaction to\na substance in the environment; symptoms include\nrash, inflammation, sneezing, itchy watery eyes,\nand runny nose.\nConjunctiva— The mucous membrane that covers\nthe white part of the eyes and lines the eyelids.\nEdema— A con

In [40]:
prompt_template = """
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
only return final answer.
Helpful answer:
"""

In [41]:
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt" : PROMPT}

In [ ]:
HF_TOKEN = "***********************"

In [33]:
os.environ["HF_TOKEN"] = HF_TOKEN
HF_LLM_BASE_URL = "https://router.huggingface.co/fireworks-ai/inference/v1"

In [37]:
import os
from openai import OpenAI
llm = ChatOpenAI(
    model_name="accounts/fireworks/models/deepseek-r1-0528", # This should match the model name expected by your HF endpoint
    openai_api_key=HF_TOKEN, # Your HF token acts as the API key here
    base_url=HF_LLM_BASE_URL,
    temperature=0.8 # Lower temperature for more factual responses, good for medical assistant
)

In [43]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=docsearch.as_retriever(search_kwargs={'k':2}),
    chain_type="stuff", # 'stuff' is common for concatenating retrieved docs directly
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs# Optional: to see which documents were retrieved
)

In [48]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query":user_input})
    print("Response:",result["result"])

Input Prompt: what are allergies


Response: <think>
Hmm, the user is asking me to explain what allergies are based on a specific context provided. They mentioned an article titled "What's New in: Asthma and Allergic Rhinitis" from Pulse magazine in 2004 and included some text from Richard Robinson and Teresa G. Odle.

Looking at the context provided, I can see a clear definition section that states: "Allergies are abnormal reactions of the immune system that occur in response to otherwise harmless substances." This seems like the most direct and authoritative answer to the user's question since it comes directly from the source they referenced.

The context also includes additional information about allergies being common medical disorders affecting many people, and some key terms like allergen, allergic reaction, and others. But since the user specifically asked for what allergies are, I should focus just on the definition part to keep the answer concise and relevant.

I'll use the exact phrasing from the definition i

Input Prompt: what is acne


Response: <think>
Hmm, the user is asking "what is acne" based on the provided context. Let me analyze this carefully.

The context clearly defines acne in the second-to-last paragraph: "Acne is a skin condition that occurs when pores or hair follicles become blocked. This blockage allows a waxy material called sebum to collect inside the pores." That's a straightforward definition presented as factual information from the Gale Encyclopedia of Medicine.

I notice the user specifically instructed to only return the helpful answer with nothing else. The context provides the complete definition in one clear sentence. There's no need to summarize or add any commentary - just extract the definition exactly as stated. 

The definition covers the core aspects: it's a skin condition caused by blocked pores/hair follicles, resulting in sebum buildup. This directly answers "what is acne" without requiring additional interpretation. Since the answer is explicitly stated in the context, I don't ne

KeyboardInterrupt: Interrupted by user